In [ ]:
!pip install numpy==2.3.1 matplotlib==3.10.3 qiskit==1.2.4 qiskit-aer==0.17.1 qiskit-algorithms==0.3.0 qiskit-nature==0.7.2 pyscf==2.9.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Import Qiskit Nature and PySCF components
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

# Import PySCF components for classical calculations
from pyscf import gto, scf, fci


def get_h2o_coords(d):
    """
    Returns H2O geometry by symmetrically stretching the O-H bonds to distance d (Å).
    The H-O-H angle is fixed at its experimental equilibrium value.
    """
    # High-precision experimental equilibrium angle from NIST CCCBDB.
    theta_rad = math.radians(104.4776)

    # Place the Oxygen at the origin. Use trigonometry to place Hydrogens.
    # The molecule lies in the xy-plane.
    h1_x = d * math.sin(theta_rad / 2)
    h1_y = d * math.cos(theta_rad / 2)

    h2_x = -d * math.sin(theta_rad / 2)
    h2_y = d * math.cos(theta_rad / 2)

    coords = [
        ["O", [0.0, 0.0, 0.0]],
        ["H", [h1_x, h1_y, 0.0]],
        ["H", [h2_x, h2_y, 0.0]],
    ]
    return coords


def get_pyscf_mol(coords, charge, spin, basis):
    """Builds a PySCF Mole object."""
    mol = gto.Mole()
    mol.atom = coords
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol


def get_qiskit_problem(coords, charge, spin, basis):
    """Generates a Qiskit Nature ElectronicStructureProblem."""
    atom_str = "; ".join([f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}" for atom in coords])
    driver = PySCFDriver(atom=atom_str, unit=DistanceUnit.ANGSTROM,
                         charge=charge, spin=spin, basis=basis)
    return driver.run()


def run_uhf_pyscf(mol):
    """Runs a classical UHF calculation with PySCF."""
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf


def run_fci_pyscf(mol):
    """Runs a classical Full CI calculation with PySCF (exact solution for the basis)."""
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci


def run_vqe(problem, mapper):
    """Runs the VQE algorithm to find the ground state energy."""
    estimator = Estimator()
    optimizer = COBYLA(maxiter=250)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)

    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
# Water is a neutral, closed-shell molecule.
basis = 'sto3g'
charge = 0
spin = 0
# Define the range of bond distances for the symmetric O-H stretch.
dists = np.linspace(0.7, 3.0, 40)

vqe_energies, uhf_energies, fci_energies = [], [], []

print("Starting calculations for H2O molecule (symmetric stretch)...")
print("-" * 75)
print(f"{'O-H Distance (Å)':<18} | {'VQE Energy (Ha)':<20} | {'UHF Energy (Ha)':<20} | {'FCI Energy (Ha)':<20}")
print("-" * 75)

# === Calculation Loop ===
for d in dists:
    coords = get_h2o_coords(d)

    vqe_energy, uhf_energy, fci_energy = np.nan, np.nan, np.nan
    try:
        # PySCF calculations
        mol = get_pyscf_mol(coords, charge, spin, basis)
        uhf_energy = run_uhf_pyscf(mol)
        fci_energy = run_fci_pyscf(mol)

        # Qiskit VQE calculation
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe(problem, mapper)

    except Exception as e:
        print(f"Calculation failed at d={d:.2f}: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)

    # Print progress
    print(f"{d:<18.2f} | {vqe_energy:<20.6f} | {uhf_energy:<20.6f} | {fci_energy:<20.6f}")

print("-" * 75)
print("Calculations complete.")

# === Plotting ===
plt.figure(figsize=(8, 6))
plt.plot(dists, vqe_energies,label='VQE (UCCSD)', color='limegreen')
plt.plot(dists, uhf_energies, label='UHF (Classical)', color='red')
plt.plot(dists, fci_energies, '--', label='FCI (Exact)', color='black')

plt.xlabel("Symmetric O-H Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("H₂O Potential Energy Curve (STO-3G basis)", fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()
